# Inadimpletes - Hackday 3° Edição

In [ ]:
#!pip install sweetviz xgboost category_encoders

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sweetviz
import pickle

from sklearn.metrics       import mean_absolute_error, mean_squared_error, f1_score
from sklearn.ensemble      import RandomForestClassifier
from sklearn.linear_model  import LinearRegression, Lasso
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, LabelEncoder
import sklearn.model_selection as ms
from xgboost                 import XGBClassifier
from category_encoders       import TargetEncoder

%matplotlib inline
sns.set_theme()
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_columns', 500)
plt.rcParams['figure.figsize'] = [25, 12]
plt.rcParams['font.size'] = 16

/home/gustavos/.conda/envs/minimal_ml/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data Ingestion

In [2]:
df_train = pd.read_csv('data/train.csv')
df_train.head()

,id_cliente,idade,saldo_atual,divida_atual,renda_anual,valor_em_investimentos,taxa_utilizacao_credito,num_emprestimos,num_contas_bancarias,num_cartoes_credito,dias_atraso_dt_venc,num_pgtos_atrasados,num_consultas_credito,taxa_juros,investe_exterior,pessoa_polit_exp,limite_adicional
0,1767,21,278.17,2577.05,24196.90,104.31,31.04,6,5,7,21,14,9,15,Não,Não,Negar
1,11920,40,268.87,2465.39,19227.38,69.86,36.92,5,8,5,40,23,10,18,Não,Não,Negar
2,8910,36,446.64,1055.29,42822.28,134.20,34.56,0,3,6,26,13,3,15,Sim,Não,Negar
3,4964,58,321.14,703.05,51786.83,297.35,31.49,0,3,7,12,7,2,1,Sim,Não,Negar
4,10100,35,428.72,891.29,44626.85,134.20,28.03,2,8,7,24,10,8,20,Sim,Não,Negar


## Data understanding

## Data dimension

In [3]:
df_train.shape

(9500, 17)

## Class distribution

In [4]:
df_train['limite_adicional'].value_counts(normalize=True)*100

Negar      84.16
Conceder   15.84
Name: limite_adicional, dtype: float64

## Data types

In [5]:
df_train.dtypes

id_cliente                   int64
idade                        int64
saldo_atual                float64
divida_atual               float64
renda_anual                float64
valor_em_investimentos     float64
taxa_utilizacao_credito    float64
num_emprestimos              int64
num_contas_bancarias         int64
num_cartoes_credito          int64
dias_atraso_dt_venc          int64
num_pgtos_atrasados          int64
num_consultas_credito        int64
taxa_juros                   int64
investe_exterior            object
pessoa_polit_exp            object
limite_adicional            object
dtype: object

## NAN?

In [6]:
df_train.isna().sum()

id_cliente                 0
idade                      0
saldo_atual                0
divida_atual               0
renda_anual                0
valor_em_investimentos     0
taxa_utilizacao_credito    0
num_emprestimos            0
num_contas_bancarias       0
num_cartoes_credito        0
dias_atraso_dt_venc        0
num_pgtos_atrasados        0
num_consultas_credito      0
taxa_juros                 0
investe_exterior           0
pessoa_polit_exp           0
limite_adicional           0
dtype: int64

## Descriptive Statistics

In [7]:
df_train.describe(include='object').T.sort_values(by='unique', ascending=False)

,count,unique,top,freq
investe_exterior,9500,2,Não,7177
pessoa_polit_exp,9500,2,Não,8917
limite_adicional,9500,2,Negar,7995


In [8]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
id_cliente,9500.00,6210.53,3603.24,1.00,3091.75,6172.50,9320.25,12500.00
idade,9500.00,126.40,685.04,18.00,28.00,38.00,47.00,8598.00
saldo_atual,9500.00,404.53,217.70,0.09,272.36,337.17,471.26,1602.04
divida_atual,9500.00,1437.09,1156.26,0.23,573.62,1175.73,1963.33,4998.07
renda_anual,9500.00,195037.27,1454273.73,10473.87,26207.90,45991.11,80449.83,23279657.98
valor_em_investimentos,9500.00,612.00,2002.44,0.00,76.08,134.20,249.18,10000.00
taxa_utilizacao_credito,9500.00,32.27,5.11,20.99,28.08,32.26,36.46,49.56
num_emprestimos,9500.00,10.86,62.56,0.00,2.00,3.00,6.00,1496.00
num_contas_bancarias,9500.00,17.55,120.98,0.00,3.00,6.00,7.00,1779.00
num_cartoes_credito,9500.00,21.43,122.68,1.00,4.00,6.00,7.00,1479.00


## Data Cleaning

In [191]:
df2 = df_train.copy()

In [192]:
def filter_outliers(df, col, factor=2):
    q3 = df[col].quantile(0.75)
    q1 = df[col].quantile(0.25)

    iqr = q3 - q1
    max_bouderie = q3 + iqr * factor
    min_bouderie = q1 - iqr * factor

    return df.loc[(df[col] >= min_bouderie) & (df[col] <= max_bouderie), :]

In [193]:
# df2 = filter_outliers(df2, 'idade')
# df2 = filter_outliers(df2, 'valor_em_investimentos')
# df2 = filter_outliers(df2, 'taxa_juros')
# df2 = filter_outliers(df2, 'renda_anual')
# df2 = filter_outliers(df2, 'num_emprestimos')
# df2 = filter_outliers(df2, 'num_cartoes_credito')
# df2 = filter_outliers(df2, 'num_contas_bancarias')
# df2 = filter_outliers(df2, 'num_consultas_credito')

## Remove useless features

In [194]:
cols_remove = ['id_cliente']
df2 = df2.drop(columns=cols_remove)

## Dataset Split

In [288]:
df_pp = df2.copy()

In [289]:
X = df_pp.drop(columns=['limite_adicional'])
y = df_pp['limite_adicional']

X_train, X_test, y_train, y_test = ms.train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [290]:
# df3 será usado para splitar Treino e Validação
df3 = pd.concat([X_train, y_train], axis = 1)



# Feature engeneering

In [291]:
def filter_outliers(df, col, factor=2):
    q3 = df[col].quantile(0.75)
    q1 = df[col].quantile(0.25)

    iqr = q3 - q1
    max_bouderie = q3 + iqr * factor
    min_bouderie = q1 - iqr * factor

    return df.loc[(df[col] >= min_bouderie) & (df[col] <= max_bouderie), :]

def filter_outliers2(df, col, factor=2):
    q3 = df[col].quantile(0.75)
    q1 = df[col].quantile(0.25)

    iqr = q3 - q1
    max_bouderie = q3 + iqr * factor
    min_bouderie = q1 - iqr * factor
    
    col_editted = df[col].map(lambda x: x if (x >= min_bouderie) and (x <= max_bouderie) else max_bouderie *2)

    return col_editted

In [292]:
# df3 = filter_outliers(df3, 'idade')
# df3 = filter_outliers(df3, 'valor_em_investimentos')
# df3 = filter_outliers(df3, 'taxa_juros')
# df3 = filter_outliers(df3, 'renda_anual')
# df3 = filter_outliers(df3, 'num_emprestimos')
# df3 = filter_outliers(df3, 'num_cartoes_credito')
# df3 = filter_outliers(df3, 'num_contas_bancarias')
# df3 = filter_outliers(df3, 'num_consultas_credito')

In [293]:
# Dividir dataset entre Treino (90%) e Validação (10%)
X = df3.drop(['limite_adicional'], axis=1).copy()
y = df3['limite_adicional'].copy()

x_train, x_val, y_train, y_val = ms.train_test_split(X, y, stratify=y, test_size=0.10, random_state = 42)

# Data Preparation

In [294]:
# x_train = filter_outliers(x_train, 'idade')
# x_train = filter_outliers(x_train, 'valor_em_investimentos')
# x_train = filter_outliers(x_train, 'taxa_juros')
# x_train = filter_outliers(x_train, 'renda_anual')
# x_train = filter_outliers(x_train, 'num_emprestimos')
# x_train = filter_outliers(x_train, 'num_cartoes_credito')
# x_train = filter_outliers(x_train, 'num_contas_bancarias')
# x_train = filter_outliers(x_train, 'num_consultas_credito')

In [295]:
x_train['idade'] = filter_outliers2(x_train, 'idade')
x_train['valor_em_investimentos'] = filter_outliers2(x_train, 'valor_em_investimentos')
x_train['taxa_juros'] = filter_outliers2(x_train, 'taxa_juros')
x_train['renda_anual'] = filter_outliers2(x_train, 'renda_anual')
x_train['num_emprestimos'] = filter_outliers2(x_train, 'num_emprestimos')
x_train['num_cartoes_credito'] = filter_outliers2(x_train, 'num_cartoes_credito')
x_train['num_contas_bancarias'] = filter_outliers2(x_train, 'num_contas_bancarias')
x_train['num_consultas_credito'] = filter_outliers2(x_train, 'num_consultas_credito')

In [296]:
# comprometimento_renda
# x_train['comprometimento_renda'] = x_train[['divida_atual', 'renda_anual']].apply(lambda x: (x['divida_atual'] / x['renda_anual'])*100, axis=1)

# x_train['comprometimento_renda'] = x_train['comprometimento_renda'].apply(lambda x:'Não' if x < 30 
#                                                                   else 'Sim')

#num_pgtos_atrasados
# x_train['num_pgtos_atrasados'] = x_train['num_pgtos_atrasados'].apply(lambda x: 
#               'Excelente' if x == 0 else 
#               'Bom' if 1 <= x <= 10 else 
#               'Regular' if 11 <= x <= 20 else 'Ruim' )

In [297]:
# Encodar features categóricas de treino

## Normalizacao
ss = StandardScaler()

## Frequency Encoding
x_train['investe_exterior'] = x_train['investe_exterior'].map(lambda x: 0.76 if x == 0.15 else 0.24)
x_train['pessoa_polit_exp'] = x_train['pessoa_polit_exp'].map(lambda x: 0.94 if x == 0.16 else 0.06)

## Rescaling
rs = RobustScaler()
mms = MinMaxScaler()

## Robust Scaler
x_train['idade'] = rs.fit_transform( x_train[['idade']].values )
x_train['saldo_atual'] = rs.fit_transform( x_train[['saldo_atual']].values )
x_train['renda_anual'] = rs.fit_transform( x_train[['renda_anual']].values )
x_train['valor_em_investimentos'] = rs.fit_transform( x_train[['valor_em_investimentos']].values )
x_train['num_emprestimos'] = rs.fit_transform( x_train[['num_emprestimos']].values )
x_train['num_contas_bancarias'] = rs.fit_transform( x_train[['num_contas_bancarias']].values )
x_train['num_cartoes_credito'] = rs.fit_transform( x_train[['num_cartoes_credito']].values )
x_train['num_pgtos_atrasados'] = rs.fit_transform( x_train[['num_pgtos_atrasados']].values )
x_train['num_consultas_credito'] = rs.fit_transform( x_train[['num_consultas_credito']].values )
x_train['taxa_juros'] = rs.fit_transform( x_train[['taxa_juros']].values )
## Min-max scaler
x_train['dias_atraso_dt_venc'] = mms.fit_transform( x_train[['dias_atraso_dt_venc']].values )
x_train['divida_atual'] = mms.fit_transform( x_train[['divida_atual']].values )
x_train['taxa_utilizacao_credito'] = ss.fit_transform( x_train[['taxa_utilizacao_credito']].values )

# # Label Enconder
# le = LabelEncoder()
# x_train['comprometimento_renda'] = le.fit_transform(x_train['comprometimento_renda'])

# pg_atrasados_dict = {'Excelente': 4, 'Bom': 3, 'Regular': 2, 'Ruim': 1}
# x_train['num_pgtos_atrasados'] = x_train['num_pgtos_atrasados'].map(pg_atrasados_dict)


# Encodar var resposta de treino
d_class = {
    'Negar': 0,
    'Conceder': 1
}

y_train = y_train.map(d_class)

In [298]:
# comprometimento_renda
# x_val['comprometimento_renda'] = x_val[['divida_atual', 'renda_anual']].apply(lambda x: (x['divida_atual'] / x['renda_anual'])*100, axis=1)

# x_val['comprometimento_renda'] = x_val['comprometimento_renda'].apply(lambda x:'Não' if x < 30 
#                                                                   else 'Sim')

# num_pgtos_atrasados
# x_val['num_pgtos_atrasados'] = x_val['num_pgtos_atrasados'].apply(lambda x: 
#               'Excelente' if x == 0 else 
#               'Bom' if 1 <= x <= 10 else 
#               'Regular' if 11 <= x <= 20 else 'Ruim' )

In [299]:
# Encodar features categóricas de treino

## Normalizacao
ss = StandardScaler()

## Frequency Encoding
x_val['investe_exterior'] = x_val['investe_exterior'].map(lambda x: 0.76 if x == 0.15 else 0.24)
x_val['pessoa_polit_exp'] = x_val['pessoa_polit_exp'].map(lambda x: 0.94 if x == 0.16 else 0.06)

## Rescaling
rs = RobustScaler()
mms = MinMaxScaler()

## Robust Scaler
x_val['idade'] = rs.fit_transform( x_val[['idade']].values )
x_val['saldo_atual'] = rs.fit_transform( x_val[['saldo_atual']].values )
x_val['renda_anual'] = rs.fit_transform( x_val[['renda_anual']].values )
x_val['valor_em_investimentos'] = rs.fit_transform( x_val[['valor_em_investimentos']].values )
x_val['num_emprestimos'] = rs.fit_transform( x_val[['num_emprestimos']].values )
x_val['num_contas_bancarias'] = rs.fit_transform( x_val[['num_contas_bancarias']].values )
x_val['num_cartoes_credito'] = rs.fit_transform( x_val[['num_cartoes_credito']].values )
x_val['num_pgtos_atrasados'] = rs.fit_transform( x_val[['num_pgtos_atrasados']].values )
x_val['num_consultas_credito'] = rs.fit_transform( x_val[['num_consultas_credito']].values )
x_val['taxa_juros'] = rs.fit_transform( x_val[['taxa_juros']].values )
## Min-max scaler
x_val['dias_atraso_dt_venc'] = mms.fit_transform( x_val[['dias_atraso_dt_venc']].values )
x_val['divida_atual'] = mms.fit_transform( x_val[['divida_atual']].values )
x_val['taxa_utilizacao_credito'] = ss.fit_transform( x_val[['taxa_utilizacao_credito']].values )

# Label Enconder
# le = LabelEncoder()
# x_val['comprometimento_renda'] = le.fit_transform(x_val['comprometimento_renda'])

# pg_atrasados_dict = {'Excelente': 4, 'Bom': 3, 'Regular': 2, 'Ruim': 1}
# x_val['num_pgtos_atrasados'] = x_val['num_pgtos_atrasados'].map(pg_atrasados_dict)

# Encodar var resposta de treino
d_class = {
    'Negar': 0,
    'Conceder': 1
}

y_val = y_val.map(d_class)

## Modeling & Evaluation

In [300]:
# Aplicação de modelos 
# Treinar XGBoost Classifier
# Model definition
model_xgboost = XGBClassifier(random_state=42)
# Model training
model_xgboost.fit(x_train, y_train[x_train.index.values])
# Model prediction
yhat_xgboost = model_xgboost.predict(x_val)

In [301]:
# Avaliação de modelos 
# Validar modelo com F1 Score (dados validação)
print(f"XGBoost - Média ponderada com F1 Score (dados validação): {round(f1_score(y_val, yhat_xgboost, average='weighted') ,3)}") #binary=default

XGBoost - Média ponderada com F1 Score (dados validação): 0.862


In [302]:
# Validar modelo com F1 Score, no método cross validation (dados validação) 
print(f"K-folds: {ms.cross_val_score(model_xgboost, x_val, y_val, scoring='f1', cv = 10)}") #'F1' métrica F1 Score p/ classificação binary
print(f"XGBoost - Média ponderada com F1 Score (dados validação): {round(ms.cross_val_score(model_xgboost, x_val, y_val, scoring='f1_weighted', cv = 10).mean() ,3)}")
# Performance manteve-se próxima com cross validation

K-folds: [0.86956522 0.53846154 0.52631579 0.47058824 0.6        0.35294118
 0.55555556 0.44444444 0.47058824 0.33333333]
XGBoost - Média ponderada com F1 Score (dados validação): 0.866


## All dataset

In [303]:
# Testar modelos
# Antes, juntar dados de treino + validação, formando um novo dataset de treino.
x_train_full = pd.concat([x_train, x_val])
y_train_full = pd.concat([y_train, y_val])

In [304]:
# # Encodar features categóricas de treino

# ## Normalizacao
# ss = StandardScaler()

# ## Frequency Encoding
# X_test['investe_exterior'] = X_test['investe_exterior'].map(lambda x: 0.76 if x == 0.15 else 0.24)
# X_test['pessoa_polit_exp'] = X_test['pessoa_polit_exp'].map(lambda x: 0.94 if x == 0.16 else 0.06)

# ## Rescaling
# rs = RobustScaler()
# mms = MinMaxScaler()

# ## Robust Scaler
# X_test['idade'] = rs.fit_transform( X_test[['idade']].values )
# X_test['saldo_atual'] = rs.fit_transform( X_test[['saldo_atual']].values )
# X_test['renda_anual'] = rs.fit_transform( X_test[['renda_anual']].values )
# X_test['valor_em_investimentos'] = rs.fit_transform( X_test[['valor_em_investimentos']].values )
# X_test['num_emprestimos'] = rs.fit_transform( X_test[['num_emprestimos']].values )
# X_test['num_contas_bancarias'] = rs.fit_transform( X_test[['num_contas_bancarias']].values )
# X_test['num_cartoes_credito'] = rs.fit_transform( X_test[['num_cartoes_credito']].values )
# X_test['num_pgtos_atrasados'] = rs.fit_transform( X_test[['num_pgtos_atrasados']].values )
# X_test['num_consultas_credito'] = rs.fit_transform( X_test[['num_consultas_credito']].values )
# X_test['taxa_juros'] = rs.fit_transform( X_test[['taxa_juros']].values )
# ## Min-max scaler
# X_test['dias_atraso_dt_venc'] = mms.fit_transform( X_test[['dias_atraso_dt_venc']].values )
# X_test['divida_atual'] = mms.fit_transform( X_test[['divida_atual']].values )
# X_test['taxa_utilizacao_credito'] = ss.fit_transform( X_test[['taxa_utilizacao_credito']].values )

# # Encodar var resposta de treino
# d_class = {
#     'Negar': 0,
#     'Conceder': 1
# }

# y_test = y_test.map(d_class)

In [305]:
# comprometimento_renda
# X_test['comprometimento_renda'] = X_test[['divida_atual', 'renda_anual']].apply(lambda x: (x['divida_atual'] / x['renda_anual'])*100, axis=1)

# X_test['comprometimento_renda'] = X_test['comprometimento_renda'].apply(lambda x:'Não' if x < 30 
#                                                                   else 'Sim')

#num_pgtos_atrasados
# X_test['num_pgtos_atrasados'] = X_test['num_pgtos_atrasados'].apply(lambda x: 
#               'Excelente' if x == 0 else 
#               'Bom' if 1 <= x <= 10 else 
#               'Regular' if 11 <= x <= 20 else 'Ruim' )

In [306]:
# Encodar features categóricas de treino

## Normalizacao
ss = StandardScaler()

## Frequency Encoding
X_test['investe_exterior'] = X_test['investe_exterior'].map(lambda x: 0.76 if x == 0.15 else 0.24)
X_test['pessoa_polit_exp'] = X_test['pessoa_polit_exp'].map(lambda x: 0.94 if x == 0.16 else 0.06)

## Rescaling
rs = RobustScaler()
mms = MinMaxScaler()

## Robust Scaler
X_test['idade'] = rs.fit_transform( X_test[['idade']].values )
X_test['saldo_atual'] = rs.fit_transform( X_test[['saldo_atual']].values )
X_test['renda_anual'] = rs.fit_transform( X_test[['renda_anual']].values )
X_test['valor_em_investimentos'] = rs.fit_transform( X_test[['valor_em_investimentos']].values )
X_test['num_emprestimos'] = rs.fit_transform( X_test[['num_emprestimos']].values )
X_test['num_contas_bancarias'] = rs.fit_transform( X_test[['num_contas_bancarias']].values )
X_test['num_cartoes_credito'] = rs.fit_transform( X_test[['num_cartoes_credito']].values )
X_test['num_pgtos_atrasados'] = rs.fit_transform( X_test[['num_pgtos_atrasados']].values )
X_test['num_consultas_credito'] = rs.fit_transform( X_test[['num_consultas_credito']].values )
X_test['taxa_juros'] = rs.fit_transform( X_test[['taxa_juros']].values )
## Min-max scaler
X_test['dias_atraso_dt_venc'] = mms.fit_transform( X_test[['dias_atraso_dt_venc']].values )
X_test['divida_atual'] = mms.fit_transform( X_test[['divida_atual']].values )
X_test['taxa_utilizacao_credito'] = ss.fit_transform( X_test[['taxa_utilizacao_credito']].values )

# # Label Enconder
# le = LabelEncoder()
# X_test['comprometimento_renda'] = le.fit_transform(X_test['comprometimento_renda'])

# pg_atrasados_dict = {'Excelente': 4, 'Bom': 3, 'Regular': 2, 'Ruim': 1}
# X_test['num_pgtos_atrasados'] = X_test['num_pgtos_atrasados'].map(pg_atrasados_dict)

# Encodar var resposta de treino
d_class = {
    'Negar': 0,
    'Conceder': 1
}

y_test = y_test.map(d_class)

In [307]:
# Treinar nova XGBoost, com o novo dataset treino
# Model definition
model_xgboost_full = XGBClassifier(random_state=42)
# Model training
model_xgboost_full.fit(x_train_full, y_train_full[x_train_full.index.values])
# Model prediction
# Passar os dados de teste separados anteriormente
yhat_xgboost_full = model_xgboost_full.predict(X_test)

In [308]:
# Validar modelo com F1 Score (dados teste)
print(f"XGBoost - Média ponderada com F1 Score (dados teste): {round(f1_score(y_test, yhat_xgboost_full, average='weighted') ,3)}") #binary=default

XGBoost - Média ponderada com F1 Score (dados teste): 0.839


Treinando com Tudo

In [281]:
# Carregar dataset teste
df_ktest_raw = pd.read_csv('data/test.csv')
x_ktest = df_ktest_raw.drop(columns=['id_cliente']).copy()
# Conferir dataset teste kaggle
x_ktest

,idade,saldo_atual,divida_atual,renda_anual,valor_em_investimentos,taxa_utilizacao_credito,num_emprestimos,num_contas_bancarias,num_cartoes_credito,dias_atraso_dt_venc,num_pgtos_atrasados,num_consultas_credito,taxa_juros,investe_exterior,pessoa_polit_exp
0,32,470.69,605.03,42688.60,104.29,24.46,1,2,4,3,4,2,6,Não,Não
1,25,288.61,943.86,43545.94,276.73,39.80,0,7,5,5,14,4,5,Não,Não
2,35,740.20,548.20,82208.45,98.67,39.96,0,4,1288,12,10,2,8,Não,Não
3,7586,298.19,1704.18,41749.69,162.44,24.45,3,8,7,30,11,5,15,Não,Não
4,49,341.55,421.43,42381.95,154.35,29.52,2,0,5,4,0,2,12,Não,Não
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,48,417.14,636.96,45532.40,94.72,25.38,2,7,3,6,8,4,19,Sim,Não
2996,43,720.34,717.79,95347.46,160.09,27.78,2,6,7,30,20,5,11,Sim,Não
2997,44,874.91,361.68,97517.22,98.88,43.31,0,2,7,5,7,1,7,Não,Não
2998,24,349.18,678.57,46943.70,131.99,35.12,4,4,4,11,4,3,3,Não,Não


In [282]:
# Encodar features categóricas de treino kaggle

## Normalizacao
ss = StandardScaler()

## Frequency Encoding
x_ktest['investe_exterior'] = x_ktest['investe_exterior'].map(lambda x: 0.76 if x == 0.15 else 0.24)
x_ktest['pessoa_polit_exp'] = x_ktest['pessoa_polit_exp'].map(lambda x: 0.94 if x == 0.16 else 0.06)

## Rescaling
rs = RobustScaler()
mms = MinMaxScaler()

## Robust Scaler
x_ktest['idade'] = rs.fit_transform( x_ktest[['idade']].values )
x_ktest['saldo_atual'] = rs.fit_transform( x_ktest[['saldo_atual']].values )
x_ktest['renda_anual'] = rs.fit_transform( x_ktest[['renda_anual']].values )
x_ktest['valor_em_investimentos'] = rs.fit_transform( x_ktest[['valor_em_investimentos']].values )
x_ktest['num_emprestimos'] = rs.fit_transform( x_ktest[['num_emprestimos']].values )
x_ktest['num_contas_bancarias'] = rs.fit_transform( x_ktest[['num_contas_bancarias']].values )
x_ktest['num_cartoes_credito'] = rs.fit_transform( x_ktest[['num_cartoes_credito']].values )
x_ktest['num_pgtos_atrasados'] = rs.fit_transform( x_ktest[['num_pgtos_atrasados']].values )
x_ktest['num_consultas_credito'] = rs.fit_transform( x_ktest[['num_consultas_credito']].values )
x_ktest['taxa_juros'] = rs.fit_transform( x_ktest[['taxa_juros']].values )
## Min-max scaler
x_ktest['dias_atraso_dt_venc'] = mms.fit_transform( x_ktest[['dias_atraso_dt_venc']].values )
x_ktest['divida_atual'] = mms.fit_transform( x_ktest[['divida_atual']].values )
x_ktest['taxa_utilizacao_credito'] = ss.fit_transform( x_ktest[['taxa_utilizacao_credito']].values )

# # Label Enconder
# le = LabelEncoder()
# x_ktest['comprometimento_renda'] = le.fit_transform(x_ktest['comprometimento_renda'])

# pg_atrasados_dict = {'Excelente': 4, 'Bom': 3, 'Regular': 2, 'Ruim': 1}
# x_ktest['num_pgtos_atrasados'] = x_ktest['num_pgtos_atrasados'].map(pg_atrasados_dict)

In [283]:
# Fazer a previsão do score usando o modelo final treinado:
yhat_xgboost_ktest = model_xgboost_full.predict(x_ktest)

In [284]:
# Preparação para submissão
# Criar dataset final
df_sub = pd.concat([df_ktest_raw['id_cliente'], pd.Series(yhat_xgboost_ktest, name='limite_adicional')], axis=1)

In [285]:
# Traduzir var resposta
lim_adi_dic_final = {0 : 'Negar', 1: 'Conceder'}
df_sub['limite_adicional'] = df_sub['limite_adicional'].map( lim_adi_dic_final )

In [286]:
# Visutalizar arquivo final para submissão ao Kaggle
df_sub[:5]

,id_cliente,limite_adicional
0,2,Conceder
1,5,Negar
2,6,Negar
3,8,Negar
4,10,Conceder


In [309]:
df_sub = df_sub.set_index('id_cliente')


In [311]:
df_sub.to_csv('data/submissions/ciclo6_v1.csv')

## Preparing submission

In [ ]:
%%time

df_test = pd.read_csv('data/test.csv')

#df_test['bank_account'] = df_test['bank_account'].map(d_class) #mapeamento class

df_test['investe_exterior'] = df_test['investe_exterior'].map(lambda x: 0.76 if x == 0.15 else 0.24)
df_test['pessoa_polit_exp'] = df_test['pessoa_polit_exp'].map(lambda x: 0.94 if x == 0.16 else 0.06)

df_test['taxa_utilizacao_credito'] = ss.fit_transform( df_test[['taxa_utilizacao_credito']].values )

rs = RobustScaler()
mms = MinMaxScaler()

## Robust Scaler
# idade
df_test['idade'] = rs.fit_transform( df_test[['idade']].values )

# saldo_atual
df_test['saldo_atual'] = rs.fit_transform( df_test[['saldo_atual']].values )

# renda_anual
df_test['renda_anual'] = rs.fit_transform( df_test[['renda_anual']].values )

# valor_em_investimentos
df_test['valor_em_investimentos'] = rs.fit_transform( df_test[['valor_em_investimentos']].values )

# num_emprestimos
df_test['num_emprestimos'] = rs.fit_transform( df_test[['num_emprestimos']].values )

# num_contas_bancarias
df_test['num_contas_bancarias'] = rs.fit_transform( df_test[['num_contas_bancarias']].values )

# num_cartoes_credito
df_test['num_cartoes_credito'] = rs.fit_transform( df_test[['num_cartoes_credito']].values )

# num_pgtos_atrasados
df_test['num_pgtos_atrasados'] = rs.fit_transform( df_test[['num_pgtos_atrasados']].values )

# num_consultas_credito
df_test['num_consultas_credito'] = rs.fit_transform( df_test[['num_consultas_credito']].values )

# taxa_juros
df_test['taxa_juros'] = rs.fit_transform( df_test[['taxa_juros']].values )

## MinMax Scaler
# dias_atraso_dt_venc
df_test['dias_atraso_dt_venc'] = mms.fit_transform( df_test[['dias_atraso_dt_venc']].values )

# divida_atual
df_test['divida_atual'] = mms.fit_transform( df_test[['divida_atual']].values )

In [ ]:
#df_ss = pd.read_csv('/kaggle/input/inclusao-financeira-na-africa/test.csv')
preds = rf.predict(df_test.drop(columns='id_cliente'))

In [ ]:
preds

In [ ]:
df_submission = pd.DataFrame()
df_submission['id_cliente'] = df_test['id_cliente']
df_submission['limite_adicional'] = preds
df_submission['limite_adicional'] = df_submission['limite_adicional'].map(lambda x: 'Conceder' if x == 1 else 'Negar')
df_submission = df_submission.set_index('id_cliente')
df_submission.head()

In [ ]:
df_submission.to_csv('data/submissions/ciclo5_v2.csv')

In [ ]:
### Ideias de melhorias
    ### 1. Testar novas formas de encoding (1d)
    ### 2. Feature engineering (2d)
    ### 3. Hyper-parameter tuning (1d)
    ### 4. Combinação de modelos